In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [2]:
EPOCH = 30
BATCH_SIZE = 64
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("Using Device:", DEVICE)

Using Device: cuda


In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data/', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data/', train=False, transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True)

In [6]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [7]:
model = VAE().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [18]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(DEVICE)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(DEVICE)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(BATCH_SIZE, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [21]:
for epoch in range(1, EPOCH + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, 20).to(DEVICE)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28), 'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 107.780594
Train Epoch: 1 [12800/60000 (21%)]	Loss: 108.607590
Train Epoch: 1 [25600/60000 (43%)]	Loss: 110.506195
Train Epoch: 1 [38400/60000 (64%)]	Loss: 108.848999
Train Epoch: 1 [51200/60000 (85%)]	Loss: 105.612373
====> Epoch: 1 Average loss: 108.1875
====> Test set loss: 107.4458
Train Epoch: 2 [0/60000 (0%)]	Loss: 111.565994
Train Epoch: 2 [12800/60000 (21%)]	Loss: 109.780762
Train Epoch: 2 [25600/60000 (43%)]	Loss: 107.499542
Train Epoch: 2 [38400/60000 (64%)]	Loss: 110.665367
Train Epoch: 2 [51200/60000 (85%)]	Loss: 108.774277
====> Epoch: 2 Average loss: 107.3425
====> Test set loss: 106.6876
Train Epoch: 3 [0/60000 (0%)]	Loss: 111.170914
Train Epoch: 3 [12800/60000 (21%)]	Loss: 105.474014
Train Epoch: 3 [25600/60000 (43%)]	Loss: 107.737511
Train Epoch: 3 [38400/60000 (64%)]	Loss: 106.684959
Train Epoch: 3 [51200/60000 (85%)]	Loss: 108.183990
====> Epoch: 3 Average loss: 106.6861
====> Test set loss: 106.2366
Train Epoch: 4 [0/60000 (0%)]	L